<a href="https://colab.research.google.com/github/isamdr86/towards-ai/blob/main/notebooks/Structured(JSON)_PDF_Data_Extraction_ir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-generativeai==0.5.4 openai tiktoken==0.7.0 llama-index-llms-gemini==0.1.10 arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.5 MB/s eta 0:00:00


In [2]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [3]:
import os
# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
# os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

### OpenAI Structured Output without `response_format`

In [4]:
from openai import OpenAI
client = OpenAI()

In [5]:
system_prompt = """
You are a helpful assistant designed to output information exclusively in JSON format. Your response should contain only raw JSON data with no additional text, explanations, or comments. Do not include backticks (`) or any code block delimiters in your response.

Always use the key `"Top10BestSellingBooks"` when listing top 10 best-selling books. Follow the specified JSON structure below:

### JSON Format Example
{
  "Top10BestSellingBooks": [
    {
      "title": "Book Title",
      "author": "Author Name",
      "yearPublished": "Year",
      "summary": "Brief summary of the book."
    },
    {
      "title": "Book Title 2",
      "author": "Author Name 2",
      "yearPublished": "Year",
      "summary": "Brief summary of the book."
    }
  ]
}
Always respond with clean JSON output that can be directly used with JSON parsers like `json.loads()`.

"""

prompt = "Give me the names of the 10 best-selling books, their authors, the year they were published, and a concise summary in JSON format"


# Making the API call
response = client.chat.completions.create(
  model="gpt-4o-2024-08-06",
  temperature = 0,
  messages=[
    {"role": "system", "content":system_prompt},
    {"role": "user", "content": prompt}
  ]
)
print(response.choices[0].message.content)

{
  "Top10BestSellingBooks": [
    {
      "title": "Don Quixote",
      "author": "Miguel de Cervantes",
      "yearPublished": "1605",
      "summary": "A Spanish novel about the adventures of a nobleman who reads so many chivalric romances that he loses his sanity and decides to become a knight-errant."
    },
    {
      "title": "A Tale of Two Cities",
      "author": "Charles Dickens",
      "yearPublished": "1859",
      "summary": "A historical novel set in London and Paris before and during the French Revolution, focusing on themes of resurrection and transformation."
    },
    {
      "title": "The Lord of the Rings",
      "author": "J.R.R. Tolkien",
      "yearPublished": "1954",
      "summary": "An epic fantasy novel that follows the quest to destroy the One Ring and defeat the Dark Lord Sauron."
    },
    {
      "title": "The Little Prince",
      "author": "Antoine de Saint-Exupéry",
      "yearPublished": "1943",
      "summary": "A philosophical tale about a young 

In [6]:
import json

result_book = json.loads(response.choices[0].message.content)

print(type(result_book))

print(result_book['Top10BestSellingBooks'][0])
print("-------------------------------------")
print(result_book['Top10BestSellingBooks'][0]['title'])

<class 'dict'>
{'title': 'Don Quixote', 'author': 'Miguel de Cervantes', 'yearPublished': '1605', 'summary': 'A Spanish novel about the adventures of a nobleman who reads so many chivalric romances that he loses his sanity and decides to become a knight-errant.'}
-------------------------------------
Don Quixote


## OpenAI Strucutred output (JSON) with `response_format`

In [7]:
from openai import OpenAI
client = OpenAI()

In [8]:
prompt = "Give me the names of the 10 best-selling books, their authors, the year they were published, and a concise summary in JSON format"

In [9]:
# The response format- JSON schema
response_format_json = {
  "type": "json_schema",
  "json_schema": {
    "name": "Top10BestSellingBooks",
    "strict": True,
    "schema": {
      "type": "object",
      "properties": {
        "Top10BestSellingBooks": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "title": { "type": "string" },
              "author": { "type": "string" },
              "yearPublished": { "type": "integer" },
              "summary": { "type": "string" }
            },

            "required": ["title", "author", "yearPublished", "summary"],
            "additionalProperties": False
          }
        }
      },
      "required": ["Top10BestSellingBooks"],
      "additionalProperties": False
    }
  }
}


In [10]:
system_prompt = """
You are a helpful assistant designed to output information exclusively in JSON format.
### JSON Format Example
{
  "Top10BestSellingBooks": [
    {
      "title": "Book Title",
      "author": "Author Name",
      "yearPublished": "Year",
      "summary": "Brief summary of the book."
    }
  ]
}

"""

In [11]:
# Making the API call
response = client.chat.completions.create(
  model="gpt-4o-2024-08-06",
  response_format=response_format_json,
  temperature = 0,
  messages=[
    {"role": "system", "content":system_prompt},
    {"role": "user", "content": prompt}
  ]
)
print(response.choices[0].message.content)

{"Top10BestSellingBooks":[{"title":"Don Quixote","author":"Miguel de Cervantes","yearPublished":1605,"summary":"A Spanish novel about the adventures of a nobleman who reads so many chivalric romances that he loses his sanity and decides to become a knight-errant."},{"title":"A Tale of Two Cities","author":"Charles Dickens","yearPublished":1859,"summary":"A historical novel set in London and Paris before and during the French Revolution, focusing on themes of resurrection and transformation."},{"title":"The Lord of the Rings","author":"J.R.R. Tolkien","yearPublished":1954,"summary":"An epic fantasy novel that follows the quest to destroy the One Ring and defeat the Dark Lord Sauron."},{"title":"The Little Prince","author":"Antoine de Saint-Exupéry","yearPublished":1943,"summary":"A philosophical tale about a young prince who travels from planet to planet, learning about life and human nature."},{"title":"Harry Potter and the Philosopher's Stone","author":"J.K. Rowling","yearPublished":1

In [12]:
print(type(response.choices[0].message.content))

<class 'str'>


In [13]:
result_book = json.loads(response.choices[0].message.content)

print(type(result_book))

print(result_book['Top10BestSellingBooks'][0])
print("-------------------------------------")
print(result_book['Top10BestSellingBooks'][0]['title'])

<class 'dict'>
{'title': 'Don Quixote', 'author': 'Miguel de Cervantes', 'yearPublished': 1605, 'summary': 'A Spanish novel about the adventures of a nobleman who reads so many chivalric romances that he loses his sanity and decides to become a knight-errant.'}
-------------------------------------
Don Quixote


## Using Pydantic

In [16]:
from pydantic import BaseModel
from typing import List

class Book(BaseModel):
    title: str
    author: str
    yearPublished: int
    summary: str

class Top10BestSellingBooks(BaseModel):
    books: List[Book]

In [19]:
response = client.beta.chat.completions.parse(
    model="gpt-4o",
    response_format=Top10BestSellingBooks,
    temperature = 0,
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "Give me the top 10 best-selling books of all time."}
    ]
)

In [21]:
print(type(response.choices[0].message.content))

<class 'str'>


In [24]:
result_book = json.loads(response.choices[0].message.content)

print(type(result_book))
print(result_book)

print(result_book['books'][0])
print("-------------------------------------")
print(result_book['books'][0]['title'])

<class 'dict'>
{'books': [{'title': 'Don Quixote', 'author': 'Miguel de Cervantes', 'yearPublished': 1605, 'summary': 'A Spanish novel that follows the adventures of a nobleman who reads so many chivalric romances that he loses his sanity and decides to become a knight-errant.'}, {'title': 'A Tale of Two Cities', 'author': 'Charles Dickens', 'yearPublished': 1859, 'summary': 'A historical novel set in London and Paris before and during the French Revolution, focusing on themes of resurrection and transformation.'}, {'title': 'The Lord of the Rings', 'author': 'J.R.R. Tolkien', 'yearPublished': 1954, 'summary': 'An epic fantasy novel that follows the quest to destroy the One Ring and defeat the Dark Lord Sauron.'}, {'title': 'The Little Prince', 'author': 'Antoine de Saint-Exupéry', 'yearPublished': 1943, 'summary': 'A novella that tells the story of a young prince who travels from planet to planet, learning about life and human nature.'}, {'title': "Harry Potter and the Philosopher's S

## Strucutred output from PDF + OpenAI + pdf2images

In [14]:
!pip install -q -U pdf2image
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (772 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [15]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="rag_research_paper.zip",repo_type="dataset",local_dir="/content")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


rag_research_paper.zip:   0%|          | 0.00/7.51M [00:00<?, ?B/s]

In [25]:
!unzip /content/rag_research_paper.zip -d /content/

Archive:  /content/rag_research_paper.zip
   creating: /content/rag_research_paper/
  inflating: /content/rag_research_paper/2405.07437v2.pdf  
  inflating: /content/rag_research_paper/2407.01219v1.pdf  
  inflating: /content/rag_research_paper/2407.07858v1.pdf  
  inflating: /content/rag_research_paper/2407.08223v1.pdf  
  inflating: /content/rag_research_paper/2407.16833v1.pdf  
  inflating: /content/rag_research_paper/2407.21712v1.pdf  
  inflating: /content/rag_research_paper/2408.08067v2.pdf  
  inflating: /content/rag_research_paper/2408.08921v1.pdf  


In [26]:
import os
from pdf2image import convert_from_path

pdf_directory = "/content/rag_research_paper"
output_dir = "/content/pages"
os.makedirs(output_dir, exist_ok=True)

pages_png = []

for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, pdf_file)

        convert = convert_from_path(pdf_path, use_pdftocairo=True)

        pdf_output_dir = os.path.join(output_dir, os.path.splitext(pdf_file)[0])
        os.makedirs(pdf_output_dir, exist_ok=True)

        for page_num, image in enumerate(convert):
            page_filename = f"page-{str(page_num + 1).zfill(3)}.png"
            full_path = os.path.join(pdf_output_dir, page_filename)
            image.save(full_path)

            pages_png.append(full_path)

print(pages_png)

['/content/pages/2408.08921v1/page-001.png', '/content/pages/2408.08921v1/page-002.png', '/content/pages/2408.08921v1/page-003.png', '/content/pages/2408.08921v1/page-004.png', '/content/pages/2408.08921v1/page-005.png', '/content/pages/2408.08921v1/page-006.png', '/content/pages/2408.08921v1/page-007.png', '/content/pages/2408.08921v1/page-008.png', '/content/pages/2408.08921v1/page-009.png', '/content/pages/2408.08921v1/page-010.png', '/content/pages/2408.08921v1/page-011.png', '/content/pages/2408.08921v1/page-012.png', '/content/pages/2408.08921v1/page-013.png', '/content/pages/2408.08921v1/page-014.png', '/content/pages/2408.08921v1/page-015.png', '/content/pages/2408.08921v1/page-016.png', '/content/pages/2408.08921v1/page-017.png', '/content/pages/2408.08921v1/page-018.png', '/content/pages/2408.08921v1/page-019.png', '/content/pages/2408.08921v1/page-020.png', '/content/pages/2408.08921v1/page-021.png', '/content/pages/2408.08921v1/page-022.png', '/content/pages/2408.08921v1/pa

In [27]:
from io import BytesIO
import base64
import json

# Function to encode the image

def encode_image(image_path):

  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [29]:
system_instruction_prompt ="""
You are an expert in extracting structured data from research paper images.

Task Description:
Extract comprehensive information from PDF research paper images, including all headlines, content, and visual elements.
Preserve complete information without fragmentation.

Must Follow Guideline: Extract all text and information accurately from each image provided. Organize content into multiple
JSON objects when appropriate, based on the amount and type of content. Each JSON should clearly reflect distinct content
sections for streamlined analysis

Content Requirements:
1. Missing Headlines
- If no visible headline exists, generate appropriate ones based on content
- Group related content under these generated headlines

2. Visual Elements
For figures, graphs, tables, and architectures:
- Extract title/caption
- Describe main trends and comparisons
- Detail architecture designs
- Include related insights from surrounding text

3. Text Processing
- Extract complete sentences without summarization
- Maintain original detail level
- Merge fragmented content logically
- Preserve all technical information

Required output Format (JSON):
[
{
    "source": "Extract complete arXiv ID including prefix (e.g., arXiv:2405.07437v2).
               Verify ID accuracy multiple times. if there is no Arxiv ID return None",

    "name": "Extract or generate all headlines and subheadlines (e.g., Abstract,
            Introduction, Methods, etc). Include section titles and subsection headings.",

    "content": "For each section:
                - Complete text content
                - Visual element descriptions
                - Figure/graph details:
                  * Title/caption
                  * Description
                  * Key trends/comparisons
                  * Architecture details
                  * Related insights"
},
]

Key Guidelines:
- Extract exact content without summarization
- Ensure accuracy in complex technical details
- Maintain logical content organization
- Include complete visual element analysis
"""

In [28]:
# The response format- JSON schema
json_response_format = {
  "type": "json_schema",
  "json_schema": {
    "name": "research_paper_data",
    "strict": True,
    "schema": {
      "type": "object",
      "properties": {
        "research_paper_data": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
                "name": { "type": "string" },
                "source": { "type": "string" },
                "content": { "type": "string"},

            },

            "required": ["name", "source", "content"],
            "additionalProperties": False
          }
        },
      },
      "required": ["research_paper_data"],
      "additionalProperties": False
    }
  }
}


In [30]:
import arxiv
import re

def arxiv_extraction(arxiv_id):
  client = arxiv.Client()
  search = arxiv.Search(id_list=re.findall(r'(\d{4}\.\d{5}|\w+(?:-\w+)?/\d{7})', arxiv_id), max_results=1)
  results = client.results(search)

  for result in results:
    return result.title, result.pdf_url

In [ ]:
import json
from openai import OpenAI
client = OpenAI()

desc = []


for page in pages_png:
  # Getting the base64
  base64_image = encode_image(page)

  try:

    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        response_format = json_response_format,
        temperature = 0,
        messages= [
              {"role": "system","content":system_instruction_prompt},
              {"role": "user","content": [{"type": "text", "text": "Extract the content from this research paper image."},
                                          {"type": "image_url","image_url": {"url":f"data:image/jpeg;base64,{base64_image}",
                                                                              "detail": "high"}}
                                          ]
                  }
                  ],
      )

    if response.choices[0].message.content is None:
      continue

    result = json.loads(response.choices[0].message.content)

    if 'page-001' in page:
      # Or You can use the Image path to extract the Arxiv Research paper ID.
      research_paper_id = result['research_paper_data'][0]['source']
      research_paper_title, research_paper_url = arxiv_extraction(research_paper_id)

      for i in range(len(result['research_paper_data'])):
        result['research_paper_data'][i]['source'] = research_paper_id
        result['research_paper_data'][i]['name'] = research_paper_title +":"+ result['research_paper_data'][i]['name']
        result['research_paper_data'][i]['url'] = research_paper_url

    if 'page-001' not in page:
      for i in range(len(result['research_paper_data'])):
        result['research_paper_data'][i]['source'] = research_paper_id
        result['research_paper_data'][i]['name'] = research_paper_title +":"+ result['research_paper_data'][i]['name']
        result['research_paper_data'][i]['url'] = research_paper_url

    desc.extend(result['research_paper_data'])

  except Exception as e:
    print(response.choices[0].finish_reason)
    print(f"Skipping {page}... error: {e}")
    break

In [ ]:
len(desc)

In [ ]:
print("Content research paper title and Headline :",desc[0]['name'],"\n")
print("Content :",desc[0]['content'],"\n")
print("Source :",desc[0]['source'],"\n")
print("URL :",desc[0]['url'])

In [ ]:
desc[1]

In [ ]:
source=[]
for i in range(len(desc)):
  source.append(desc[i]['source'])

source=list(set(source))
source



---



In [ ]:
# To incldue Generated response with collected content
import json
with open("/content/ai_tutor_500.jsonl", "r") as file:
    result = [json.loads(line) for line in file]

result[1]

In [ ]:
result[1]['tokens']

In [ ]:
import tiktoken
import uuid

def token_encoding_and_doc_id(content):
  encoding = tiktoken.get_encoding("cl100k_base")
  tokens = encoding.encode(content)

  num_tokens = len(tokens)
  doc_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, content))

  return num_tokens, doc_id

for i in range(len(desc)):
  desc[i]['tokens'], desc[i]['doc_id'] = token_encoding_and_doc_id(desc[i]['content'])

for i in range(len(result)):
  result[i]['tokens'], result[i]['doc_id'] = token_encoding_and_doc_id(result[i]['content'])

In [ ]:
ai_tutor_knowledge = result + desc

In [ ]:
ai_tutor_knowledge[850]

In [ ]:
ai_tutor_knowledge = [d for d in ai_tutor_knowledge if d['tokens'] > 100]

In [ ]:
with open('/content/ai_tutor_knowledge.jsonl', 'w') as f:
    for item in ai_tutor_knowledge:
        f.write(json.dumps(item) + '\n')

print("JSONL file saved successfully!")

In [ ]:
from google.colab import files
files.download('/content/ai_tutor_knowledge.jsonl')

------------